In [10]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import sys
import importlib.util
import subprocess

# Install required packages if missing
for dep in ["nglview", "ase"]:
    if importlib.util.find_spec(dep) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", dep])

# Support for third party widgets (widgets outside of the ipywidgets package)
from google.colab import output
output.enable_custom_widget_manager()

In [12]:
import nglview as nv
from ase.spacegroup import crystal

# Create a sample crystal structure
atoms = crystal('C', [(0, 0, 0)], spacegroup=227, cellpar=[3.567, 3.567, 3.567, 90, 90, 90])

# Show it
nv.show_ase(atoms)


NGLWidget()

# Structure Refinement: LBCO, HRPT

This example demonstrates how to use the EasyDiffraction API in a
simplified, user-friendly manner that closely follows the GUI workflow for a
Rietveld refinement of La0.5Ba0.5CoO3 crystal structure using constant
wavelength neutron powder diffraction data from HRPT at PSI.

It is intended for users with minimal programming experience who want to
learn how to perform standard crystal structure fitting using diffraction
data. This script covers creating a project, adding sample models and
experiments, performing analysis, and refining parameters.

Only a single import of `easydiffraction` is required, and all operations are
performed through high-level components of the `project` object, such as
`project.sample_models`, `project.experiments`, and `project.analysis`.
The `project` object is the main container for all information.

## Import Library

In [ ]:
import easydiffraction as ed

## Step 1: Create a Project

This section explains how to create a project and define its metadata.

#### Create Project

In [ ]:
project = ed.Project(name='lbco_hrpt')

#### Set Project Metadata

In [ ]:
project.info.title = 'La0.5Ba0.5CoO3 at HRPT@PSI'
project.info.description = """This project demonstrates a standard
refinement of La0.5Ba0.5CoO3, which crystallizes in a perovskite-type
structure, using neutron powder diffraction data collected in constant
wavelength mode at the HRPT diffractometer (PSI)."""

#### Show Project Metadata as CIF

In [ ]:
project.info.show_as_cif()

#### Save Project

When saving the project for the first time, you need to specify the
directory path. In the example below, the project is saved to a
temporary location defined by the system.

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

#### Set Up Data Plotter

Show supported plotting engines.

In [ ]:
project.plotter.show_supported_engines()

Show current plotting configuration.

In [ ]:
project.plotter.show_config()

Set plotting engine.

In [ ]:
project.plotter.engine = 'plotly'

## Step 2: Define Sample Model

This section shows how to add sample models and modify their parameters.

#### Add Sample Model

In [ ]:
project.sample_models.add(name='lbco')

#### Show Defined Sample Models

Show the names of the models added. These names are used to access the
model using the syntax: `project.sample_models['model_name']`. All model
parameters can be accessed via the `project` object.

In [ ]:
project.sample_models.show_names()

#### Set Space Group

Modify the default space group parameters.

In [ ]:
project.sample_models['lbco'].space_group.name_h_m = 'P m -3 m'
project.sample_models['lbco'].space_group.it_coordinate_system_code = '1'

#### Set Unit Cell

Modify the default unit cell parameters.

In [ ]:
project.sample_models['lbco'].cell.length_a = 3.88

#### Set Atom Sites

Add atom sites to the sample model.

In [ ]:
project.sample_models['lbco'].atom_sites.add(
    label='La',
    type_symbol='La',
    fract_x=0,
    fract_y=0,
    fract_z=0,
    wyckoff_letter='a',
    b_iso=0.5,
    occupancy=0.5,
)
project.sample_models['lbco'].atom_sites.add(
    label='Ba',
    type_symbol='Ba',
    fract_x=0,
    fract_y=0,
    fract_z=0,
    wyckoff_letter='a',
    b_iso=0.5,
    occupancy=0.5,
)
project.sample_models['lbco'].atom_sites.add(
    label='Co',
    type_symbol='Co',
    fract_x=0.5,
    fract_y=0.5,
    fract_z=0.5,
    wyckoff_letter='b',
    b_iso=0.5,
)
project.sample_models['lbco'].atom_sites.add(
    label='O',
    type_symbol='O',
    fract_x=0,
    fract_y=0.5,
    fract_z=0.5,
    wyckoff_letter='c',
    b_iso=0.5,
)

#### Apply Symmetry Constraints

In [ ]:
project.sample_models['lbco'].apply_symmetry_constraints()

#### Show Sample Model as CIF

In [ ]:
project.sample_models['lbco'].show_as_cif()

#### Show Sample Model Structure

In [ ]:
project.sample_models['lbco'].show_structure()

#### Save Project State

Save the project state after adding the sample model. This ensures
that all changes are stored and can be accessed later. The project
state is saved in the directory specified during project creation.

In [ ]:
project.save()

## Step 3: Define Experiment

This section shows how to add experiments, configure their parameters, and
link the sample models defined in the previous step.

#### Download Measured Data

Download the data file from the EasyDiffraction repository on GitHub.

In [ ]:
ed.download_from_repository('hrpt_lbco.xye', destination='data')

#### Add Diffraction Experiment

In [ ]:
project.experiments.add(
    name='hrpt',
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='neutron',
    data_path='data/hrpt_lbco.xye',
)

#### Show Defined Experiments

In [ ]:
project.experiments.show_names()

#### Show Measured Data

In [ ]:
project.plot_meas(expt_name='hrpt')

#### Set Instrument

Modify the default instrument parameters.

In [ ]:
project.experiments['hrpt'].instrument.setup_wavelength = 1.494
project.experiments['hrpt'].instrument.calib_twotheta_offset = 0.6

#### Set Peak Profile

Show supported peak profile types.

In [ ]:
project.experiments['hrpt'].show_supported_peak_profile_types()

Show the current peak profile type.

In [ ]:
project.experiments['hrpt'].show_current_peak_profile_type()

Select the desired peak profile type.

In [ ]:
project.experiments['hrpt'].peak_profile_type = 'pseudo-voigt'

Modify default peak profile parameters.

In [ ]:
project.experiments['hrpt'].peak.broad_gauss_u = 0.1
project.experiments['hrpt'].peak.broad_gauss_v = -0.1
project.experiments['hrpt'].peak.broad_gauss_w = 0.1
project.experiments['hrpt'].peak.broad_lorentz_x = 0
project.experiments['hrpt'].peak.broad_lorentz_y = 0.1

#### Set Background

Show supported background types.

In [ ]:
project.experiments['hrpt'].show_supported_background_types()

Show current background type.

In [ ]:
project.experiments['hrpt'].show_current_background_type()

Select the desired background type.

In [ ]:
project.experiments['hrpt'].background_type = 'line-segment'

Add background points.

In [ ]:
project.experiments['hrpt'].background.add(x=10, y=170)
project.experiments['hrpt'].background.add(x=30, y=170)
project.experiments['hrpt'].background.add(x=50, y=170)
project.experiments['hrpt'].background.add(x=110, y=170)
project.experiments['hrpt'].background.add(x=165, y=170)

Show current background points.

In [ ]:
project.experiments['hrpt'].background.show()

#### Set Linked Phases

Link the sample model defined in the previous step to the experiment.

In [ ]:
project.experiments['hrpt'].linked_phases.add(id='lbco', scale=10.0)

#### Show Experiment as CIF

In [ ]:
project.experiments['hrpt'].show_as_cif()

#### Save Project State

In [ ]:
project.save()

## Step 4: Perform Analysis

This section explains the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

Show supported calculation engines.

In [ ]:
project.analysis.show_supported_calculators()

Show current calculation engine.

In [ ]:
project.analysis.show_current_calculator()

Select the desired calculation engine.

In [ ]:
project.analysis.current_calculator = 'cryspy'

#### Show Calculated Data

In [ ]:
project.plot_calc(expt_name='hrpt')

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Show Parameters

Show all parameters of the project.

In [ ]:
project.analysis.show_all_params()

Show all fittable parameters.

In [ ]:
project.analysis.show_fittable_params()

Show only free parameters.

In [ ]:
project.analysis.show_free_params()

Show how to access parameters in the code.

In [ ]:
project.analysis.how_to_access_parameters()

#### Set Fit Mode

Show supported fit modes.

In [ ]:
project.analysis.show_available_fit_modes()

Show current fit mode.

In [ ]:
project.analysis.show_current_fit_mode()

Select desired fit mode.

In [ ]:
project.analysis.fit_mode = 'single'

#### Set Minimizer

Show supported fitting engines.

In [ ]:
project.analysis.show_available_minimizers()

Show current fitting engine.

In [ ]:
project.analysis.show_current_minimizer()

Select desired fitting engine.

In [ ]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

### Perform Fit 1/5

Set sample model parameters to be refined.

In [ ]:
project.sample_models['lbco'].cell.length_a.free = True

Set experiment parameters to be refined.

In [ ]:
project.experiments['hrpt'].linked_phases['lbco'].scale.free = True
project.experiments['hrpt'].instrument.calib_twotheta_offset.free = True
project.experiments['hrpt'].background['10'].y.free = True
project.experiments['hrpt'].background['30'].y.free = True
project.experiments['hrpt'].background['50'].y.free = True
project.experiments['hrpt'].background['110'].y.free = True
project.experiments['hrpt'].background['165'].y.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Save Project State

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [ ]:
project.experiments['hrpt'].peak.broad_gauss_u.free = True
project.experiments['hrpt'].peak.broad_gauss_v.free = True
project.experiments['hrpt'].peak.broad_gauss_w.free = True
project.experiments['hrpt'].peak.broad_lorentz_y.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Save Project State

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

### Perform Fit 3/5

Set more parameters to be refined.

In [ ]:
project.sample_models['lbco'].atom_sites['La'].b_iso.free = True
project.sample_models['lbco'].atom_sites['Ba'].b_iso.free = True
project.sample_models['lbco'].atom_sites['Co'].b_iso.free = True
project.sample_models['lbco'].atom_sites['O'].b_iso.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Save Project State

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

### Perform Fit 4/5

#### Set Constraints

Set aliases for parameters.

In [ ]:
project.analysis.aliases.add(label='biso_La', param_uid=project.sample_models['lbco'].atom_sites['La'].b_iso.uid)
project.analysis.aliases.add(label='biso_Ba', param_uid=project.sample_models['lbco'].atom_sites['Ba'].b_iso.uid)

Set constraints.

In [ ]:
project.analysis.constraints.add(lhs_alias='biso_Ba', rhs_expr='biso_La')

Show defined constraints.

In [ ]:
project.analysis.show_constraints()

Show free parameters before applying constraints.

In [ ]:
project.analysis.show_free_params()

Apply constraints.

In [ ]:
project.analysis.apply_constraints()

Show free parameters after applying constraints.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Save Project State

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

### Perform Fit 5/5

#### Set Constraints

Set more aliases for parameters.

In [ ]:
project.analysis.aliases.add(
    label='occ_La',
    param_uid=project.sample_models['lbco'].atom_sites['La'].occupancy.uid,
)
project.analysis.aliases.add(
    label='occ_Ba',
    param_uid=project.sample_models['lbco'].atom_sites['Ba'].occupancy.uid,
)

Set more constraints.

In [ ]:
project.analysis.constraints.add(
    lhs_alias='occ_Ba',
    rhs_expr='1 - occ_La',
)

Show defined constraints.

In [ ]:
project.analysis.show_constraints()

Apply constraints.

In [ ]:
project.analysis.apply_constraints()

Set sample model parameters to be refined.

In [ ]:
project.sample_models['lbco'].atom_sites['La'].occupancy.free = True

Show free parameters after selection.

In [ ]:
project.analysis.show_free_params()

#### Run Fitting

In [ ]:
project.analysis.fit()

#### Plot Measured vs Calculated

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', show_residual=True)

In [ ]:
project.plot_meas_vs_calc(expt_name='hrpt', x_min=38, x_max=41, show_residual=True)

#### Save Project State

In [ ]:
project.save_as(dir_path='lbco_hrpt', temporary=True)

## Step 5: Summary

This final section shows how to review the results of the analysis.

#### Show Project Summary

In [ ]:
project.summary.show_report()